In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
#from fake_useragent import UserAgent
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import time
import os
import pandas as pd
import csv
import json

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get("https://www.datanyze.com/market-share")
driver.implicitly_wait(10)

P_SP_D_categories = []
templist = []

P_SP_D_categories = driver.find_elements(By.XPATH, "//li/a")
for element in P_SP_D_categories:
    sub_cat = element.text
    href = element.get_attribute('href')
    parent_cat = element.find_element(By.XPATH, './/parent::li//parent::ul//preceding-sibling::div').text
    #print(f"{parent_cat}, {sub_cat}, {href}")
    table_dict = {'parent_cat': parent_cat, 'sub_cat': sub_cat, 'domain': href}
    templist.append(table_dict)
    df = pd.DataFrame(templist)
df.to_csv('Datanyze_data_test.csv')


driver.quit()

In [80]:
#function to click-more until there is no click-more.
#see more button - //*[contains(@class, 'see-more') and contains(@class, 'see-more-text')]
def click_more_button():
    click_more = True
    try:
        while click_more:
            time.sleep(1)
            #//button[contains(@class, 'm-component-footer--loader') and contains(@class, 'm-button')]
            element = driver.find_element_by_xpath(("//*[contains(@class, 'see-more') and contains(@class, 'see-more-text')]"))
            if element:
                element.click()
            else:
                click_more = False
    except:
        return None

#Click on 'more' option functionality on sub_category page.
def click_more_tech(driver):
    try:
        #/td[2]/div/tech-description/p/span
        #//span[@class='show-more' and text()='More']
        more_button = driver.find_elements_by_xpath("//span[@class='show-more' and text()='More']")
        #print(len(more_button))
        for more in more_button:
            driver.execute_script("arguments[0].click();", more)
    except Exception as e:
        print(e)
        return None

def get_element_data(element):
    description = ""
    try:
        description = element.text
    except Exception as e:
        return description
    finally: 
        return description
    
def restart_wifi():
    # Restart wifi in case capterra blocks
    os.system("networksetup -setairportpower airport off")
    time.sleep(10)
    os.system("networksetup -setairportpower airport on")
    time.sleep(10)
    


df_main = pd.DataFrame()

        
P_S_D_data = pd.read_csv('/Users/shagarwal/Datanyze_webScraping/Datanyze_data.csv', header=0)
for index,row in P_S_D_data.iterrows():
    templist = []
    parent_cat = row[1]
    sub_cat = row[2]
    href = row[3]
    
    try:
        firefox_options = FirefoxOptions()
        driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
        #driver = webdriver.Firefox(Firefox)
        
        driver.get(href)
        driver.maximize_window()
        driver.set_page_load_timeout(30)
        click_more_button()
        click_more_tech(driver)
        
        all_row_data = driver.find_elements(By.XPATH, "//table//tbody//tr")
        print(f"*****************:> {parent_cat}, {sub_cat} <:*****************")
        for row_data in all_row_data:
            #ranking_data =row_data.find_element(By.XPATH, ".//td").text
            try:
                ranking_data = get_element_data(row_data.find_element(By.XPATH, ".//td"))
            except:
                ranking_data = ""

            #print(ranking_data)

            #Technology - //table//tbody//tr//td//div//a[text()='QuickBooks']
            #technology = row_data.find_element(By.XPATH, ".//td//div//a").text
            technology = get_element_data(row_data.find_element(By.XPATH, ".//td//div//a"))
            try:
                technology = get_element_data(row_data.find_element(By.XPATH, ".//td//div//a"))
            except:
                technology = ""
            #print(technology)

            #technology-description//p[contains(@class, 'tech-description')]
            #///table/tbody/tr/td[2]/div/tech-description/p
            #tech_description = row_data.find_element(By.XPATH, "./td[2]/div/tech-description/p").text
            try:
                tech_description = get_element_data(row_data.find_element(By.XPATH, "./td[2]/div/tech-description/p"))
            except:
                tech_description == ""
            #print(tech_description)

            #vendor name and domain//table//tbody//tr//td[3]//div//a
            #vendor_name = row_data.find_element(By.XPATH, ".//td[3]//div//a").text
            vendor_name = get_element_data(row_data.find_element(By.XPATH, ".//td[3]//div//a"))
            try:
                vendor_name = get_element_data(row_data.find_element(By.XPATH, ".//td[3]//div//a"))
            except:
                vendor_name = ""

            #print(vendor_name)
            try:
                vendor_href = row_data.find_element(By.XPATH, ".//td[3]//div//a").get_attribute('href')
            except:
                vendor_href = ''

            #companies using - //table//tbody//tr//td[4]
            #company_using = row_data.find_element(By.XPATH, ".//td[4]").text
            try:
                company_using = get_element_data(row_data.find_element(By.XPATH, ".//td[4]"))
            except:
                company_using = ''
            #print(company_using)

            #market share - //table//tbody//tr//td[5]
            #market_share = row_data.find_element(By.XPATH, ".//td[5]").text
            try:
                market_share = get_element_data(row_data.find_element(By.XPATH, ".//td[5]"))
            except:
                market_share = ''
            #print(market_share)
            print(f"{technology}")
            table_list = [parent_cat, sub_cat, href, ranking_data, technology, tech_description, vendor_name, vendor_href, company_using, market_share]
            templist.append(table_list)
    except Exception as e:
        print(e)
        time.sleep(5)
    else:
        df_temp = pd.DataFrame(templist)
        df_temp.to_csv(f"/Users/shagarwal/Datanyze_webScraping/Results/{sub_cat}_{datetime.now()}.csv", index=False)
        driver.quit()
        time.sleep(5)
        
        



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/shagarwal/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache
/var/folders/19/rxnzmy3s3451t4_8x5tm5vdr0000gp/T/ipykernel_72878/1389143437.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = driver.find_element_by_xpath(("//*[contains(@class, 'see-more') and contains(@class, 'see-more-text')]"))
/var/folders/19/rxnzmy3s3451t4_8x5tm5vdr0000gp/T/ipykernel_72878/1389143437.py:23: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  more_button = driver.find_elements_by_xpath("//span[@class='show-more' and text()='More']")


*****************:> Web Tools and Plugins, Social Login <:*****************
Linkedin JavaScript SDK
Facebook Login
LinkedIn Login
Google Sign-In for Websites
Twitter Sign-In
OneAll
Janrain Social Login
Gigya Registration as a Service
SocialPlus
Gigya Login
Yahoo Login
Apptha Magento Social Login
Intuit Sign-In
Twitch OAuth2
CoffeeBean Social-ID




====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/shagarwal/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache


*****************:> Web Tools and Plugins, Social Sharing <:*****************
Tweet Button
AddThis
ShareThis
AddToAny
Simple Share Buttons
Shareaholic
bShare
Hupso Share Buttons
Po.st
Uptolike
VKontakte
E-MAILiT
XING Share Button
NiftyButtons
Lockerz Share
Slideshare
Janrain Social Sharing
Gigya Share Bar
OdnaKnopka
IFTTT
OCLC ILLiad
JiaThis
Zenback
Addvocate
Ollco
Socialite.js
SoClever




====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/shagarwal/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache


...

*****************:> Web Tools and Plugins, Web Fonts <:*****************
Google Font API
Adobe Typekit
cufon
Fonts.com Web Fonts
Adobe Edge Web Fonts
FontDeck
Webtype
Useso Fonts
Extensis WebINK
myfonts
Goldshell FlaX




====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/shagarwal/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache


*****************:> Web Tools and Plugins, Website Builders <:*****************
Wix
Squarespace
GoCentral
Weebly
Duda
1&1 MyWebsite
Microsoft Frontpage
Jimdo
Adobe Muse
Webflow
Webs
Mono Solutions
Netlify Drop (formerly BitBalloon)
CM4All
Blogger
Mobirise
Google Sites
Homestead
Yola
WebSite X5
Sitezoogle
Adobe Portfolio
Showit
SiteBuilder.com
Shogun
iWeb
Yahoo SiteBuilder
Tumblr
Tilda
Cargo
Odoo Website Builder
Strikingly
WebStarts
Webnode
SpaceCraft
WebsPlanet
Zeta Producer
MotoCMS
ITC Insurance Website Builder
ASI ESP Websites
IM Creator
Mr Site
Tripod
Format
Hemsida24
PhotoBiz
TypePad
LiveEdit
Flickity
BaseKit
Kotisivukone
FASO
PagineGialle Visual Site
HostGator Website Builder
Webydo
Pagevamp
Medium
SITE123
Altervista
Livedoor Blog
Virb
GoDaddy Website Website Builder (Website Tonight)
Bandcamp for Labels
Cocolog Nifty
Bazel
Nethouse
Moonfruit
CCH Site Builder
Showit Desktop
Microsoft Expression Studio
React Static
yclas
Mahara
Mozello
Setup.ru
Svbtle
SiteMaker Pro
Zinnia
WebMatrix



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/shagarwal/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache


*****************:> Web Tools and Plugins, Widgets <:*****************
Facebook SDK
Facebook Page Plugin
Embedly
Facebook Message-Us Plugin
POWr
YARPP
Snapwidget
Skype Contact me button
Mixcloud API
Bravenet Web Tools
FeedWind
Bandsintown Website Plugins
GetSiteControl
Spreaker Widget
BibleGateway Verse of the Day
ReverbNation WIdgets
Pocket for Publishers
LinkWithin
Vine Embed
Etsy Mini
Feedjit
Tutiempo
TradePending Snap
CustomerLobby
Scroll to Top Button
Qt
Twitter Hashtag Button
Goodreads Widgets
Thickbox
Tweetmeme
Instagram Feed for Wix
Booking.com Review Widget
Google Matched Content
Box Embed
El Tiempo Widget
Climatempo
Fancy Tools
Zipwhip Click-to-Text
Websta Widgets
CryptoCompare Widget
esports
Wheatherbug Weather Widgets
QuoteMedia
WooCommerce Moneris Gateway
nRelate
eLend Solutions
PhotoSnack
MakeMyDeal widget
Reddit Widget
TK
GreatSchools School Finder Widget
Boat Load Puzzles Free Daily Crossword Web Gadget
Drive It Now
ProMax Instant Screen
Panoramio Widget API
Graphiq
Plu